In [156]:
import typing as t
import operator as op
import subprocess as sp
import glob
import os
from concurrent.futures import ThreadPoolExecutor, ProcessPoolExecutor
from itertools import chain

import tqdm
import pandas as pd
from fn import F
from Bio import SearchIO, SeqIO

pd.set_option('display.max_rows', 1000)
pd.set_option('display.width', 3000)
%matplotlib inline

In [32]:
os.chdir("/home/gladkov2/storage/metagenome/bact_4x/all_procca_faa")

In [47]:
scan_files = [
    'hm_pr_01.faa.txt',
    'hm_pr_02.faa.txt',
    'hm_pr_03.faa.txt',
    'hm_pr_04.faa.txt'
]
samples = ["c1",
          "c2",
          "c3",
          "c4"]

scans = [
    list(SearchIO.parse(fname, 'hmmer3-text')) for fname in scan_files
]

In [48]:
hmmscan_annotation_keys = [
    'query', 'model', 'description', 'bitscore', 
    'evalue', 'domains_expected', 'domains_observed'
]

def extract_query_annotations(query) -> pd.DataFrame:
    query_id = query.id
    hits = query.hits
    hit_ids = [hit.id for hit in hits]
    hit_descriptions = [hit.description for hit in hits]
    hit_scores = [hit.bitscore for hit in hits]
    hit_evalues = [hit.evalue for hit in hits]
    domains_expected = [hit.domain_exp_num for hit in hits]
    domains_observed = [hit.domain_obs_num for hit in hits]
    records = [
        [query_id, hit_id, hit_descr, hit_score, hit_eval, dom_exp, dom_obs]
        for hit_id, hit_descr, hit_score, hit_eval, dom_exp, dom_obs, in 
        zip(hit_ids, hit_descriptions, hit_scores, hit_evalues,
            domains_expected, domains_observed)
    ]
    return pd.DataFrame(records, columns=hmmscan_annotation_keys)


def extract_hmmscan_annotations(sample_id, sample_queries) -> pd.DataFrame:
    empty = pd.DataFrame(
        data=[['NA', 'NA', 'NA', 0, 99999.0, 0.0, 0.0]], 
        columns=hmmscan_annotation_keys
    )
    hmmscan = (
        F(map, extract_query_annotations)
        >> list
        >> (lambda dfs: pd.concat(dfs) if dfs else empty)
        >> (lambda df: df.reset_index(drop=True))
    )(sample_queries)
    return pd.concat([
        pd.Series(sample_id, index=hmmscan.index, name='sample'),
        hmmscan
    ], axis=1)

In [52]:
scan_annotations = [
    extract_hmmscan_annotations(id_, scan) for id_, scan in zip(samples, scans)
]

In [53]:
scan_annotations_concat = (
    pd.concat(scan_annotations)
    .reset_index(drop=True)
    .sort_values(['sample', 'description'])
)
scan_annotations_concat.sort_values("query", inplace = True)
scan_annotations_concat.drop_duplicates(subset ="query", 
                     keep = False, inplace = True) 

In [1]:
scan_annotations_concat.to_csv('/home/gladkov/storage/glyc/scan_concat.tsv', sep='\t', index=False)


NameError: name 'scan_annotations_concat' is not defined

In [247]:
scan_annotations_concat = pd.read_csv("annotations.tsv", delimiter="\t")

In [248]:
scan_annotations_concat

,sample,query,model,description,bitscore,evalue,domains_expected,domains_observed
0,c2,BODNEAOK_00260,PF02156.15,Glycosyl hydrolase family 26,167.8,4.900000e-52,1.0,1
1,c2,BODNEAOK_01207,PF01915.22,Glycosyl hydrolase family 3 C-terminal domain,79.2,5.000000e-25,1.2,1
2,c2,BODNEAOK_01768,PF00933.21,Glycosyl hydrolase family 3 N terminal domain,132.4,2.800000e-41,1.0,1
3,c2,BODNEAOK_01866,PF02156.15,Glycosyl hydrolase family 26,100.4,1.600000e-31,1.0,1
4,c2,BODNEAOK_02255,PF00933.21,Glycosyl hydrolase family 3 N terminal domain,191.3,3.300000e-59,1.3,1
...,...,...,...,...,...,...,...,...
1011,c1,ODGLODFF_95671,PF01270.17,Glycosyl hydrolases family 8,66.7,1.900000e-21,2.1,2
1012,c1,ODGLODFF_95687,PF00457.17,Glycosyl hydrolases family 11,136.1,1.200000e-42,1.1,1
1013,c1,ODGLODFF_95695,PF00150.18,Cellulase (glycosyl hydrolase family 5),150.1,1.100000e-46,1.0,1
1014,c1,ODGLODFF_96168,PF00722.21,Glycosyl hydrolases family 16,74.8,7.000000e-24,1.0,1


In [68]:
scan_annotations_concat.query('sample == "c2"')

,sample,query,model,description,bitscore,evalue,domains_expected,domains_observed
249,c2,BODNEAOK_00260,PF02156.15,Glycosyl hydrolase family 26,167.8,4.900000e-52,1.0,1
250,c2,BODNEAOK_01207,PF01915.22,Glycosyl hydrolase family 3 C-terminal domain,79.2,5.000000e-25,1.2,1
251,c2,BODNEAOK_01768,PF00933.21,Glycosyl hydrolase family 3 N terminal domain,132.4,2.800000e-41,1.0,1
252,c2,BODNEAOK_01866,PF02156.15,Glycosyl hydrolase family 26,100.4,1.600000e-31,1.0,1
253,c2,BODNEAOK_02255,PF00933.21,Glycosyl hydrolase family 3 N terminal domain,191.3,3.300000e-59,1.3,1
254,c2,BODNEAOK_02483,PF00331.20,Glycosyl hydrolase family 10,336.5,1.800000e-103,1.1,1
255,c2,BODNEAOK_02484,PF00331.20,Glycosyl hydrolase family 10,104.3,8.800000e-33,1.0,1
256,c2,BODNEAOK_02486,PF00331.20,Glycosyl hydrolase family 10,102.7,2.700000e-32,1.2,1
257,c2,BODNEAOK_02489,PF03512.13,Glycosyl hydrolase family 52,422.1,2.900000e-129,1.1,1
258,c2,BODNEAOK_02501,PF04616.14,Glycosyl hydrolases family 43,341.5,4.800000e-105,1.8,2


In [69]:
scan_annotations_concat.query('sample == "c3"')

,sample,query,model,description,bitscore,evalue,domains_expected,domains_observed
610,c3,EHPFJGHK_01371,PF00150.18,Cellulase (glycosyl hydrolase family 5),66.8,2.700000e-21,1.1,1
611,c3,EHPFJGHK_03429,PF00331.20,Glycosyl hydrolase family 10,121.4,5.600000e-38,2.0,2
612,c3,EHPFJGHK_03808,PF00933.21,Glycosyl hydrolase family 3 N terminal domain,124.8,6.000000e-39,1.0,1
615,c3,EHPFJGHK_05470,PF00150.18,Cellulase (glycosyl hydrolase family 5),85.5,5.300000e-27,2.1,2
618,c3,EHPFJGHK_07954,PF00933.21,Glycosyl hydrolase family 3 N terminal domain,66.4,3.500000e-21,1.0,1
619,c3,EHPFJGHK_07955,PF01915.22,Glycosyl hydrolase family 3 C-terminal domain,76.2,4.100000e-24,2.0,1
620,c3,EHPFJGHK_08338,PF00933.21,Glycosyl hydrolase family 3 N terminal domain,237.9,2.200000e-73,1.1,1
621,c3,EHPFJGHK_08650,PF06964.12,Alpha-L-arabinofuranosidase C-terminal domain,121.5,4.700000e-38,1.0,1
622,c3,EHPFJGHK_08653,PF00331.20,Glycosyl hydrolase family 10,121.2,6.400000e-38,1.1,1
623,c3,EHPFJGHK_08654,PF01915.22,Glycosyl hydrolase family 3 C-terminal domain,169.6,1.000000e-52,1.2,1


In [74]:
scan_annotations_concat.groupby("sample").count()

,query,model,description,bitscore,evalue,domains_expected,domains_observed
sample,,,,,,,
c1,226,226,226,226,226,226,226
c2,326,326,326,326,326,326,326
c3,205,205,205,205,205,205,205
c4,259,259,259,259,259,259,259


In [75]:
scan_annotations_concat['description'].value_counts()

Glycosyl hydrolases family 43                               219
Glycosyl hydrolase family 3 N terminal domain               178
Glycosyl hydrolase family 10                                 99
Glycosyl hydrolase family 3 C-terminal domain                87
Cellulase (glycosyl hydrolase family 5)                      77
Glycosyl hydrolase family 9                                  60
Alpha-L-arabinofuranosidase C-terminal domain                57
Glycosyl hydrolases family 16                                35
Glycosyl hydrolase family 115                                34
Glycosyl hydrolases family 8                                 28
Glycosyl hydrolase family 26                                 23
Glycosyl hydrolases family 6                                 19
Glycosyl hydrolase family 67 middle domain                   17
Glycosyl hydrolases family 11                                17
Gylcosyl hydrolase family 115 C-terminal domain              16
Glycosyl hydrolases family 39           

In [136]:
df_sum = scan_annotations_concat.groupby("sample")['description'].value_counts(normalize=True) * 100

In [265]:
df_sum_noper = scan_annotations_concat.groupby("sample")['description'].value_counts()
df_sum_noper.to_csv('sum_noper.tsv', sep='\t', index=True)

In [139]:
df_sum.to_csv('sum.tsv', sep='\t', index=True)

In [81]:
selected_queries = set(scan_annotations_concat['query'])
proteins = [
    SeqIO.parse('pr_01.faa', 'fasta'),
    SeqIO.parse('pr_02.faa', 'fasta'),
    SeqIO.parse('pr_03.faa', 'fasta'),
    SeqIO.parse('pr_01.faa', 'fasta')
]
for samp, prots in zip(samples, proteins):
    SeqIO.write((seq for seq in prots if seq.id in selected_queries), f'{samp}.faa', 'fasta')

In [194]:
kid01 = pd.read_csv("kaiju/kid_01.txt", delimiter="\t", header=None)
kid02 = pd.read_csv("kaiju/kid_02.txt", delimiter="\t", header=None)
kid03 = pd.read_csv("kaiju/kid_03.txt", delimiter="\t", header=None)
kid04 = pd.read_csv("kaiju/kid_04.txt", delimiter="\t", header=None)
df01 = scan_annotations_concat.query('sample == "c1"').assign(taxa = kid01[2])
df02 = scan_annotations_concat.query('sample == "c2"').assign(taxa = kid02[2])
df03 = scan_annotations_concat.query('sample == "c3"').assign(taxa = kid03[2])
df04 = scan_annotations_concat.query('sample == "c4"').assign(taxa = kid04[2])

In [272]:
kid_fin = kid01.append(kid02).append(kid03).append(kid04)
kid_fin.columns = ["query", "taxa1", "taxa"]
kid_fin['taxa'] = kid_fin.taxa.str.split(';').apply(lambda x: ';'.join(x[::-1]))
len(kid_fin)/len(scan_annotations_concat)

0.4251968503937008

In [270]:
kid_fin['taxa'].value_counts()

Opitutus terrae;Opitutus;Opitutaceae;Opitutales;Opitutae;Verrucomicrobia;Bacteria                                                                           43
Devosia riboflavina;Devosia;Hyphomicrobiaceae;Rhizobiales;Alphaproteobacteria;Proteobacteria;Bacteria                                                       21
Asticcacaulis sp. YBE204;Asticcacaulis;Caulobacteraceae;Caulobacterales;Alphaproteobacteria;Proteobacteria;Bacteria                                         17
Sporocytophaga myxococcoides;Sporocytophaga;Cytophagaceae;Cytophagales;Cytophagia;Bacteroidetes;Bacteria                                                    16
Devosia sp. LC5;Devosia;Hyphomicrobiaceae;Rhizobiales;Alphaproteobacteria;Proteobacteria;Bacteria                                                           14
Asticcacaulis excentricus;Asticcacaulis;Caulobacteraceae;Caulobacterales;Alphaproteobacteria;Proteobacteria;Bacteria                                        11
Devosia sp. 17-2-E-8;Devosia;Hyphomicrobiaceae